In [6]:
import psycopg2
import pandas as pd

# 🔌 Conectar ao banco de dados
conn = psycopg2.connect(
    dbname="mimiciv",
    user="desafio_mimic",
    password="desafio_mimic",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

# ⚙️ Tabelas temporárias: internação e mortalidade
sql_script = """
CREATE TEMP TABLE ultima_uti AS
SELECT *
FROM (
    SELECT 
        i.subject_id,
        i.hadm_id,
        i.stay_id,
        i.intime,
        i.outtime,
        p.dod,
        a.admittime,
        a.dischtime,
        a.hospital_expire_flag,
        ROW_NUMBER() OVER (PARTITION BY i.subject_id ORDER BY i.outtime DESC) AS rn
    FROM mimiciv_icu.icustays i
    JOIN mimiciv_hosp.admissions a ON i.hadm_id = a.hadm_id
    JOIN mimiciv_hosp.patients p ON i.subject_id = p.subject_id
) t
WHERE rn = 1;

CREATE TEMP TABLE pacientes_mortos AS
SELECT
  subject_id,
  stay_id,
  CASE
    WHEN dod IS NOT NULL AND dod <= outtime THEN 'Óbito intra-UTI'
    WHEN dod IS NOT NULL AND dod > outtime AND dod <= dischtime THEN 'Óbito hospitalar pós-UTI'
    WHEN dod IS NOT NULL AND dod > dischtime AND dod <= dischtime + INTERVAL '1 year' THEN 'Óbito em até 1 ano após alta hospitalar'
    ELSE 'Sobrevivente (≥ 1 ano pós-alta)'
  END AS categoria
FROM ultima_uti;
"""
cursor.execute("ROLLBACK")
cursor.execute(sql_script)
conn.commit()

# 📥 Target
query_target = """
SELECT subject_id, stay_id,
  CASE WHEN categoria = 'Sobrevivente (≥ 1 ano pós-alta)' THEN 0 ELSE 1 END AS target
FROM pacientes_mortos;
"""
df_modelagem = pd.read_sql(query_target, conn)

# 📥 Demográficos
query_demo = """
SELECT DISTINCT p.subject_id, p.gender, p.anchor_age AS idade
FROM mimiciv_hosp.patients p
JOIN pacientes_mortos pm ON p.subject_id = pm.subject_id;
"""
df_demo = pd.read_sql(query_demo, conn)

# 📥 Charlson
query_charlson = """
SELECT subject_id, charlson_comorbidity_index AS charlson
FROM mimiciv_derived.charlson
WHERE subject_id IN (SELECT subject_id FROM pacientes_mortos);
"""
df_charlson = pd.read_sql(query_charlson, conn).drop_duplicates('subject_id')

# 🔗 Merge demográficos + Charlson
df_modelagem = df_modelagem.merge(df_demo, on="subject_id", how="left")
df_modelagem = df_modelagem.merge(df_charlson, on="subject_id", how="left")
df_modelagem["charlson"] = df_modelagem["charlson"].fillna(0)

# 📥 Diagnósticos mais frequentes
query_top_diag = """
SELECT d.icd_code, COUNT(*) AS freq
FROM mimiciv_hosp.diagnoses_icd d
JOIN (SELECT DISTINCT subject_id, hadm_id FROM mimiciv_icu.icustays) icu
  ON d.subject_id = icu.subject_id AND d.hadm_id = icu.hadm_id
JOIN pacientes_mortos pm ON d.subject_id = pm.subject_id
WHERE pm.categoria != 'Sobrevivente (≥ 1 ano pós-alta)'
GROUP BY d.icd_code
ORDER BY freq DESC
LIMIT 20;
"""
df_top_diag = pd.read_sql(query_top_diag, conn)
top_icds = df_top_diag["icd_code"].tolist()

# 📥 Diagnósticos por paciente
query_dx = """
SELECT DISTINCT subject_id, icd_code
FROM mimiciv_hosp.diagnoses_icd
WHERE subject_id IN (SELECT subject_id FROM pacientes_mortos);
"""
df_dx = pd.read_sql(query_dx, conn)

# 🔢 Criar flags binárias para diagnósticos
df_dx_flags = df_dx[["subject_id"]].drop_duplicates().copy()
for icd in top_icds:
    col = f"dx_{icd}"
    df_dx_flags[col] = df_dx["icd_code"].eq(icd).groupby(df_dx["subject_id"]).transform("max")
df_dx_flags = df_dx_flags.drop_duplicates("subject_id")
df_modelagem = df_modelagem.merge(df_dx_flags, on="subject_id", how="left")

# 🔧 Ajuste final de colunas dx_ (tira espaços e transforma em int)
df_modelagem.columns = df_modelagem.columns.str.strip()
dx_cols = [col for col in df_modelagem.columns if col.startswith("dx_")]
df_modelagem[dx_cols] = df_modelagem[dx_cols].fillna(0).astype(int)

# 📥 Vasopressores
query_vaso = """
WITH vaso_flag AS (
    SELECT
        v.stay_id,
        MAX(CASE WHEN v.dopamine IS NOT NULL THEN 1 ELSE 0 END) AS vaso_dopamine,
        MAX(CASE WHEN v.epinephrine IS NOT NULL THEN 1 ELSE 0 END) AS vaso_epinephrine,
        MAX(CASE WHEN v.norepinephrine IS NOT NULL THEN 1 ELSE 0 END) AS vaso_norepi,
        MAX(CASE WHEN v.phenylephrine IS NOT NULL THEN 1 ELSE 0 END) AS vaso_pheny,
        MAX(CASE WHEN v.dobutamine IS NOT NULL THEN 1 ELSE 0 END) AS vaso_dobutamine,
        MAX(CASE WHEN v.milrinone IS NOT NULL THEN 1 ELSE 0 END) AS vaso_milrinone
    FROM mimiciv_derived.vasoactive_agent v
    GROUP BY v.stay_id
),
vasopressin_flag AS (
    SELECT DISTINCT stay_id, 1 AS vaso_vasopressin
    FROM mimiciv_derived.vasopressin
)
SELECT 
    COALESCE(vf.stay_id, vp.stay_id) AS stay_id,
    COALESCE(vf.vaso_dopamine, 0) AS vaso_dopamine,
    COALESCE(vf.vaso_epinephrine, 0) AS vaso_epinephrine,
    COALESCE(vf.vaso_norepi, 0) AS vaso_norepi,
    COALESCE(vf.vaso_pheny, 0) AS vaso_pheny,
    COALESCE(vf.vaso_dobutamine, 0) AS vaso_dobutamine,
    COALESCE(vf.vaso_milrinone, 0) AS vaso_milrinone,
    COALESCE(vp.vaso_vasopressin, 0) AS vaso_vasopressin
FROM vaso_flag vf
FULL OUTER JOIN vasopressin_flag vp ON vf.stay_id = vp.stay_id;
"""
df_vaso = pd.read_sql(query_vaso, conn)
df_modelagem = df_modelagem.merge(df_vaso, on="stay_id", how="left")

# 📥 Ventilação Mecânica
query_vm = """
WITH ventilacao_mec AS (
    SELECT DISTINCT stay_id, 1 AS usou_vm
    FROM mimiciv_derived.ventilation
    WHERE ventilation_status IN ('InvasiveVent', 'NonInvasiveVent', 'SupplementalOxygen', 'HFNC', 'Tracheostomy')
)
SELECT 
    u.stay_id,
    COALESCE(v.usou_vm, 0) AS usou_vm
FROM ultima_uti u
LEFT JOIN ventilacao_mec v ON u.stay_id = v.stay_id;
"""
df_vm = pd.read_sql(query_vm, conn)
df_modelagem = df_modelagem.merge(df_vm, on="stay_id", how="left")

# 📥 TSR
query_tsr = """
SELECT DISTINCT stay_id, 1 AS usou_tsr
FROM mimiciv_derived.first_day_rrt
WHERE dialysis_present = 1;
"""
df_tsr = pd.read_sql(query_tsr, conn)
df_modelagem = df_modelagem.merge(df_tsr, on="stay_id", how="left")

# 📥 SAPSII, OASIS, SOFA
query_scores = """
SELECT 
    p.subject_id,
    MAX(s.sapsii) AS sapsii,
    MAX(s.sapsii_prob) AS sapsii_prob,
    MAX(o.oasis) AS oasis,
    MAX(o.oasis_prob) AS oasis_prob,
    MAX(f.sofa_24hours) AS sofa_24hours
FROM mimiciv_icu.icustays p
JOIN mimiciv_derived.sapsii s ON p.stay_id = s.stay_id
JOIN mimiciv_derived.oasis o ON p.stay_id = o.stay_id
JOIN mimiciv_derived.sofa f ON p.stay_id = f.stay_id
JOIN pacientes_mortos pm ON p.stay_id = pm.stay_id
GROUP BY p.subject_id;
"""
df_scores = pd.read_sql(query_scores, conn)
df_modelagem = df_modelagem.merge(df_scores, on="subject_id", how="left")

# 🔧 Final: preencher NaNs
df_modelagem.fillna(0, inplace=True)

# 💾 Exportar CSV
df_modelagem.to_csv("dataset_modelagem_completo.csv", index=False)
print("📁 Dataset final salvo como: dataset_modelagem_completo.csv")

/tmp/ipykernel_144069/4122043005.py:58: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_modelagem = pd.read_sql(query_target, conn)
/tmp/ipykernel_144069/4122043005.py:66: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_demo = pd.read_sql(query_demo, conn)
/tmp/ipykernel_144069/4122043005.py:74: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_charlson = pd.read_sql(query_charlson, conn).drop_duplicates('subject_id')
/tmp/ipykernel_144069/4122043005.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/c

📁 Dataset final salvo como: dataset_modelagem_completo.csv


In [1]:
import psycopg2
import pandas as pd

# 🔌 Conectar ao banco de dados
conn = psycopg2.connect(
    dbname="mimiciv",
    user="desafio_mimic",
    password="desafio_mimic",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

# ⚙️ Tabelas temporárias: internação e mortalidade
sql_script = """
CREATE TEMP TABLE ultima_uti AS
SELECT *
FROM (
    SELECT 
        i.subject_id,
        i.hadm_id,
        i.stay_id,
        i.intime,
        i.outtime,
        p.dod,
        a.admittime,
        a.dischtime,
        a.hospital_expire_flag,
        ROW_NUMBER() OVER (PARTITION BY i.subject_id ORDER BY i.outtime DESC) AS rn
    FROM mimiciv_icu.icustays i
    JOIN mimiciv_hosp.admissions a ON i.hadm_id = a.hadm_id
    JOIN mimiciv_hosp.patients p ON i.subject_id = p.subject_id
) t
WHERE rn = 1;

CREATE TEMP TABLE pacientes_mortos AS
SELECT
  subject_id,
  stay_id,
  CASE
    WHEN dod IS NOT NULL AND dod <= outtime THEN 'Óbito intra-UTI'
    WHEN dod IS NOT NULL AND dod > outtime AND dod <= dischtime THEN 'Óbito hospitalar pós-UTI'
    WHEN dod IS NOT NULL AND dod > dischtime AND dod <= dischtime + INTERVAL '1 year' THEN 'Óbito em até 1 ano após alta hospitalar'
    ELSE 'Sobrevivente (≥ 1 ano pós-alta)'
  END AS categoria
FROM ultima_uti;
"""
cursor.execute("ROLLBACK")
cursor.execute(sql_script)
conn.commit()

# 📥 Exames: Obter a média, variância e o último exame
query_exames = """
WITH exames_filtrados AS (
    SELECT
        e.subject_id,
        e.valuenum,
        e.charttime,
        ROW_NUMBER() OVER (PARTITION BY e.subject_id ORDER BY e.charttime DESC) AS rn
    FROM mimiciv_icu.chartevents e
    JOIN pacientes_mortos pm ON e.stay_id = pm.stay_id
    WHERE e.valuenum IS NOT NULL
),
estatisticas_exames AS (
    SELECT 
        subject_id,
        AVG(valuenum) AS media,
        VARIANCE(valuenum) AS variancia
    FROM exames_filtrados
    GROUP BY subject_id
),
ultimo_exame AS (
    SELECT 
        subject_id,
        valuenum AS ultimo_valor
    FROM exames_filtrados
    WHERE rn = 1
)
SELECT 
    e.subject_id,
    e.media,
    e.variancia,
    u.ultimo_valor
FROM estatisticas_exames e
JOIN ultimo_exame u ON e.subject_id = u.subject_id;
"""

# Carregar os resultados da consulta SQL
df_exames = pd.read_sql(query_exames, conn)

# 📥 Target
query_target = """
SELECT subject_id, stay_id,
  CASE WHEN categoria = 'Sobrevivente (≥ 1 ano pós-alta)' THEN 0 ELSE 1 END AS target
FROM pacientes_mortos;
"""
df_modelagem = pd.read_sql(query_target, conn)

# 📥 Demográficos
query_demo = """
SELECT DISTINCT p.subject_id, p.gender, p.anchor_age AS idade
FROM mimiciv_hosp.patients p
JOIN pacientes_mortos pm ON p.subject_id = pm.subject_id;
"""
df_demo = pd.read_sql(query_demo, conn)

# 📥 Charlson
query_charlson = """
SELECT subject_id, charlson_comorbidity_index AS charlson
FROM mimiciv_derived.charlson
WHERE subject_id IN (SELECT subject_id FROM pacientes_mortos);
"""
df_charlson = pd.read_sql(query_charlson, conn).drop_duplicates('subject_id')

# 🔗 Merge demográficos + Charlson
df_modelagem = df_modelagem.merge(df_demo, on="subject_id", how="left")
df_modelagem = df_modelagem.merge(df_charlson, on="subject_id", how="left")
df_modelagem["charlson"] = df_modelagem["charlson"].fillna(0)

# 📥 Diagnósticos mais frequentes
query_top_diag = """
SELECT d.icd_code, COUNT(*) AS freq
FROM mimiciv_hosp.diagnoses_icd d
JOIN (SELECT DISTINCT subject_id, hadm_id FROM mimiciv_icu.icustays) icu
  ON d.subject_id = icu.subject_id AND d.hadm_id = icu.hadm_id
JOIN pacientes_mortos pm ON d.subject_id = pm.subject_id
WHERE pm.categoria != 'Sobrevivente (≥ 1 ano pós-alta)'
GROUP BY d.icd_code
ORDER BY freq DESC
LIMIT 20;
"""
df_top_diag = pd.read_sql(query_top_diag, conn)
top_icds = df_top_diag["icd_code"].tolist()

# 📥 Diagnósticos por paciente
query_dx = """
SELECT DISTINCT subject_id, icd_code
FROM mimiciv_hosp.diagnoses_icd
WHERE subject_id IN (SELECT subject_id FROM pacientes_mortos);
"""
df_dx = pd.read_sql(query_dx, conn)

# 🔢 Criar flags binárias para diagnósticos
df_dx_flags = df_dx[["subject_id"]].drop_duplicates().copy()
for icd in top_icds:
    col = f"dx_{icd}"
    df_dx_flags[col] = df_dx["icd_code"].eq(icd).groupby(df_dx["subject_id"]).transform("max")
df_dx_flags = df_dx_flags.drop_duplicates("subject_id")
df_modelagem = df_modelagem.merge(df_dx_flags, on="subject_id", how="left")

# 🔧 Ajuste final de colunas dx_ (tira espaços e transforma em int)
df_modelagem.columns = df_modelagem.columns.str.strip()
dx_cols = [col for col in df_modelagem.columns if col.startswith("dx_")]
df_modelagem[dx_cols] = df_modelagem[dx_cols].fillna(0).astype(int)

# 🔧 Merge de exames com o dataset final
df_modelagem = df_modelagem.merge(df_exames, on="subject_id", how="left")

# 📥 Vasopressores
query_vaso = """
WITH vaso_flag AS (
    SELECT
        v.stay_id,
        MAX(CASE WHEN v.dopamine IS NOT NULL THEN 1 ELSE 0 END) AS vaso_dopamine,
        MAX(CASE WHEN v.epinephrine IS NOT NULL THEN 1 ELSE 0 END) AS vaso_epinephrine,
        MAX(CASE WHEN v.norepinephrine IS NOT NULL THEN 1 ELSE 0 END) AS vaso_norepi,
        MAX(CASE WHEN v.phenylephrine IS NOT NULL THEN 1 ELSE 0 END) AS vaso_pheny,
        MAX(CASE WHEN v.dobutamine IS NOT NULL THEN 1 ELSE 0 END) AS vaso_dobutamine,
        MAX(CASE WHEN v.milrinone IS NOT NULL THEN 1 ELSE 0 END) AS vaso_milrinone
    FROM mimiciv_derived.vasoactive_agent v
    GROUP BY v.stay_id
),
vasopressin_flag AS (
    SELECT DISTINCT stay_id, 1 AS vaso_vasopressin
    FROM mimiciv_derived.vasopressin
)
SELECT 
    COALESCE(vf.stay_id, vp.stay_id) AS stay_id,
    COALESCE(vf.vaso_dopamine, 0) AS vaso_dopamine,
    COALESCE(vf.vaso_epinephrine, 0) AS vaso_epinephrine,
    COALESCE(vf.vaso_norepi, 0) AS vaso_norepi,
    COALESCE(vf.vaso_pheny, 0) AS vaso_pheny,
    COALESCE(vf.vaso_dobutamine, 0) AS vaso_dobutamine,
    COALESCE(vf.vaso_milrinone, 0) AS vaso_milrinone,
    COALESCE(vp.vaso_vasopressin, 0) AS vaso_vasopressin
FROM vaso_flag vf
FULL OUTER JOIN vasopressin_flag vp ON vf.stay_id = vp.stay_id;
"""
df_vaso = pd.read_sql(query_vaso, conn)
df_modelagem = df_modelagem.merge(df_vaso, on="stay_id", how="left")

# 📥 Ventilação Mecânica
query_vm = """
WITH ventilacao_mec AS (
    SELECT DISTINCT stay_id, 1 AS usou_vm
    FROM mimiciv_derived.ventilation
    WHERE ventilation_status IN ('InvasiveVent', 'NonInvasiveVent', 'SupplementalOxygen', 'HFNC', 'Tracheostomy')
)
SELECT 
    u.stay_id,
    COALESCE(v.usou_vm, 0) AS usou_vm
FROM ultima_uti u
LEFT JOIN ventilacao_mec v ON u.stay_id = v.stay_id;
"""
df_vm = pd.read_sql(query_vm, conn)
df_modelagem = df_modelagem.merge(df_vm, on="stay_id", how="left")

# 📥 TSR
query_tsr = """
SELECT DISTINCT stay_id, 1 AS usou_tsr
FROM mimiciv_derived.first_day_rrt
WHERE dialysis_present = 1;
"""
df_tsr = pd.read_sql(query_tsr, conn)
df_modelagem = df_modelagem.merge(df_tsr, on="stay_id", how="left")

# 📥 SAPSII, OASIS, SOFA
query_scores = """
SELECT 
    p.subject_id,
    MAX(s.sapsii) AS sapsii,
    MAX(s.sapsii_prob) AS sapsii_prob,
    MAX(o.oasis) AS oasis,
    MAX(o.oasis_prob) AS oasis_prob,
    MAX(f.sofa_24hours) AS sofa_24hours
FROM mimiciv_icu.icustays p
JOIN mimiciv_derived.sapsii s ON p.stay_id = s.stay_id
JOIN mimiciv_derived.oasis o ON p.stay_id = o.stay_id
JOIN mimiciv_derived.sofa f ON p.stay_id = f.stay_id
JOIN pacientes_mortos pm ON p.stay_id = pm.stay_id
GROUP BY p.subject_id;
"""
df_scores = pd.read_sql(query_scores, conn)
df_modelagem = df_modelagem.merge(df_scores, on="subject_id", how="left")

# 🔧 Final: preencher NaNs
df_modelagem.fillna(0, inplace=True)

# Remover duplicatas no DataFrame com base no 'subject_id'
df_modelagem = df_modelagem.drop_duplicates(subset=['subject_id'])

# 💾 Exportar CSV
df_modelagem.to_csv("dataset_modelagem_completo.csv", index=False)
print("📁 Dataset final salvo como: dataset_modelagem_completo.csv")

/tmp/ipykernel_167189/3764901025.py:89: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_exames = pd.read_sql(query_exames, conn)
/tmp/ipykernel_167189/3764901025.py:97: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_modelagem = pd.read_sql(query_target, conn)
/tmp/ipykernel_167189/3764901025.py:105: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_demo = pd.read_sql(query_demo, conn)
/tmp/ipykernel_167189/3764901025.py:113: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string UR

📁 Dataset final salvo como: dataset_modelagem_completo.csv


In [2]:
import psycopg2
import pandas as pd

# 🔌 Conectar ao banco de dados
conn = psycopg2.connect(
    dbname="mimiciv",
    user="desafio_mimic",
    password="desafio_mimic",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

# ⚙️ Tabelas temporárias: internação e mortalidade
sql_script = """
CREATE TEMP TABLE ultima_uti AS
SELECT *
FROM (
    SELECT 
        i.subject_id,
        i.hadm_id,
        i.stay_id,
        i.intime,
        i.outtime,
        p.dod,
        a.admittime,
        a.dischtime,
        a.hospital_expire_flag,
        ROW_NUMBER() OVER (PARTITION BY i.subject_id ORDER BY i.outtime DESC) AS rn
    FROM mimiciv_icu.icustays i
    JOIN mimiciv_hosp.admissions a ON i.hadm_id = a.hadm_id
    JOIN mimiciv_hosp.patients p ON i.subject_id = p.subject_id
) t
WHERE rn = 1;

CREATE TEMP TABLE pacientes_mortos AS
SELECT
  subject_id,
  stay_id,
  CASE
    WHEN dod IS NOT NULL AND dod <= outtime THEN 'Óbito intra-UTI'
    WHEN dod IS NOT NULL AND dod > outtime AND dod <= dischtime THEN 'Óbito hospitalar pós-UTI'
    WHEN dod IS NOT NULL AND dod > dischtime AND dod <= dischtime + INTERVAL '1 year' THEN 'Óbito em até 1 ano após alta hospitalar'
    ELSE 'Sobrevivente (≥ 1 ano pós-alta)'
  END AS categoria
FROM ultima_uti;
"""
cursor.execute("ROLLBACK")
cursor.execute(sql_script)
conn.commit()

# 📥 Exames: Obter a média, variância e o último exame
query_exames = """
WITH exames_filtrados AS (
    SELECT
        e.subject_id,
        e.valuenum,
        e.charttime,
        ROW_NUMBER() OVER (PARTITION BY e.subject_id ORDER BY e.charttime DESC) AS rn
    FROM mimiciv_icu.chartevents e
    JOIN pacientes_mortos pm ON e.stay_id = pm.stay_id
    WHERE e.valuenum IS NOT NULL
),
estatisticas_exames AS (
    SELECT 
        subject_id,
        AVG(valuenum) AS media,
        VARIANCE(valuenum) AS variancia
    FROM exames_filtrados
    GROUP BY subject_id
),
ultimo_exame AS (
    SELECT 
        subject_id,
        valuenum AS ultimo_valor
    FROM exames_filtrados
    WHERE rn = 1
)
SELECT 
    e.subject_id,
    e.media,
    e.variancia,
    u.ultimo_valor
FROM estatisticas_exames e
JOIN ultimo_exame u ON e.subject_id = u.subject_id;
"""

# Carregar os resultados da consulta SQL
df_exames = pd.read_sql(query_exames, conn)

# 📥 Target
query_target = """
SELECT subject_id, stay_id,
  CASE WHEN categoria = 'Sobrevivente (≥ 1 ano pós-alta)' THEN 0 ELSE 1 END AS target
FROM pacientes_mortos;
"""
df_modelagem = pd.read_sql(query_target, conn)

# 📥 Demográficos
query_demo = """
SELECT DISTINCT p.subject_id, p.gender, p.anchor_age AS idade
FROM mimiciv_hosp.patients p
JOIN pacientes_mortos pm ON p.subject_id = pm.subject_id;
"""
df_demo = pd.read_sql(query_demo, conn)

# 📥 Charlson
query_charlson = """
SELECT subject_id, charlson_comorbidity_index AS charlson
FROM mimiciv_derived.charlson
WHERE subject_id IN (SELECT subject_id FROM pacientes_mortos);
"""
df_charlson = pd.read_sql(query_charlson, conn).drop_duplicates('subject_id')

# 🔗 Merge demográficos + Charlson
df_modelagem = df_modelagem.merge(df_demo, on="subject_id", how="left")
df_modelagem = df_modelagem.merge(df_charlson, on="subject_id", how="left")
df_modelagem["charlson"] = df_modelagem["charlson"].fillna(0)

# 📥 Diagnósticos mais frequentes
query_top_diag = """
SELECT d.icd_code, COUNT(*) AS freq
FROM mimiciv_hosp.diagnoses_icd d
JOIN (SELECT DISTINCT subject_id, hadm_id FROM mimiciv_icu.icustays) icu
  ON d.subject_id = icu.subject_id AND d.hadm_id = icu.hadm_id
JOIN pacientes_mortos pm ON d.subject_id = pm.subject_id
WHERE pm.categoria != 'Sobrevivente (≥ 1 ano pós-alta)'
GROUP BY d.icd_code
ORDER BY freq DESC
LIMIT 20;
"""
df_top_diag = pd.read_sql(query_top_diag, conn)
top_icds = df_top_diag["icd_code"].tolist()

# 📥 Diagnósticos por paciente
query_dx = """
SELECT DISTINCT subject_id, icd_code
FROM mimiciv_hosp.diagnoses_icd
WHERE subject_id IN (SELECT subject_id FROM pacientes_mortos);
"""
df_dx = pd.read_sql(query_dx, conn)

# 🔢 Criar flags binárias para diagnósticos
df_dx_flags = df_dx[["subject_id"]].drop_duplicates().copy()
for icd in top_icds:
    col = f"dx_{icd}"
    df_dx_flags[col] = df_dx["icd_code"].eq(icd).groupby(df_dx["subject_id"]).transform("max")
df_dx_flags = df_dx_flags.drop_duplicates("subject_id")
df_modelagem = df_modelagem.merge(df_dx_flags, on="subject_id", how="left")

# 🔧 Ajuste final de colunas dx_ (tira espaços e transforma em int)
df_modelagem.columns = df_modelagem.columns.str.strip()
dx_cols = [col for col in df_modelagem.columns if col.startswith("dx_")]
df_modelagem[dx_cols] = df_modelagem[dx_cols].fillna(0).astype(int)

# 🔧 Merge de exames com o dataset final
df_modelagem = df_modelagem.merge(df_exames, on="subject_id", how="left")

# 📥 Vasopressores
query_vaso = """
WITH vaso_flag AS (
    SELECT
        v.stay_id,
        MAX(CASE WHEN v.dopamine IS NOT NULL THEN 1 ELSE 0 END) AS vaso_dopamine,
        MAX(CASE WHEN v.epinephrine IS NOT NULL THEN 1 ELSE 0 END) AS vaso_epinephrine,
        MAX(CASE WHEN v.norepinephrine IS NOT NULL THEN 1 ELSE 0 END) AS vaso_norepi,
        MAX(CASE WHEN v.phenylephrine IS NOT NULL THEN 1 ELSE 0 END) AS vaso_pheny,
        MAX(CASE WHEN v.dobutamine IS NOT NULL THEN 1 ELSE 0 END) AS vaso_dobutamine,
        MAX(CASE WHEN v.milrinone IS NOT NULL THEN 1 ELSE 0 END) AS vaso_milrinone
    FROM mimiciv_derived.vasoactive_agent v
    GROUP BY v.stay_id
),
vasopressin_flag AS (
    SELECT DISTINCT stay_id, 1 AS vaso_vasopressin
    FROM mimiciv_derived.vasopressin
)
SELECT 
    COALESCE(vf.stay_id, vp.stay_id) AS stay_id,
    COALESCE(vf.vaso_dopamine, 0) AS vaso_dopamine,
    COALESCE(vf.vaso_epinephrine, 0) AS vaso_epinephrine,
    COALESCE(vf.vaso_norepi, 0) AS vaso_norepi,
    COALESCE(vf.vaso_pheny, 0) AS vaso_pheny,
    COALESCE(vf.vaso_dobutamine, 0) AS vaso_dobutamine,
    COALESCE(vf.vaso_milrinone, 0) AS vaso_milrinone,
    COALESCE(vp.vaso_vasopressin, 0) AS vaso_vasopressin
FROM vaso_flag vf
FULL OUTER JOIN vasopressin_flag vp ON vf.stay_id = vp.stay_id;
"""
df_vaso = pd.read_sql(query_vaso, conn)
df_modelagem = df_modelagem.merge(df_vaso, on="stay_id", how="left")

# 📥 Ventilação Mecânica
query_vm = """
WITH ventilacao_mec AS (
    SELECT DISTINCT stay_id, 1 AS usou_vm
    FROM mimiciv_derived.ventilation
    WHERE ventilation_status IN ('InvasiveVent', 'NonInvasiveVent', 'SupplementalOxygen', 'HFNC', 'Tracheostomy')
)
SELECT 
    u.stay_id,
    COALESCE(v.usou_vm, 0) AS usou_vm
FROM ultima_uti u
LEFT JOIN ventilacao_mec v ON u.stay_id = v.stay_id;
"""
df_vm = pd.read_sql(query_vm, conn)
df_modelagem = df_modelagem.merge(df_vm, on="stay_id", how="left")

# 📥 TSR
query_tsr = """
SELECT DISTINCT stay_id, 1 AS usou_tsr
FROM mimiciv_derived.first_day_rrt
WHERE dialysis_present = 1;
"""
df_tsr = pd.read_sql(query_tsr, conn)
df_modelagem = df_modelagem.merge(df_tsr, on="stay_id", how="left")

# 📥 SAPSII, OASIS, SOFA
query_scores = """
SELECT 
    p.subject_id,
    MAX(s.sapsii) AS sapsii,
    MAX(s.sapsii_prob) AS sapsii_prob,
    MAX(o.oasis) AS oasis,
    MAX(o.oasis_prob) AS oasis_prob,
    MAX(f.sofa_24hours) AS sofa_24hours
FROM mimiciv_icu.icustays p
JOIN mimiciv_derived.sapsii s ON p.stay_id = s.stay_id
JOIN mimiciv_derived.oasis o ON p.stay_id = o.stay_id
JOIN mimiciv_derived.sofa f ON p.stay_id = f.stay_id
JOIN pacientes_mortos pm ON p.stay_id = pm.stay_id
GROUP BY p.subject_id;
"""
df_scores = pd.read_sql(query_scores, conn)
df_modelagem = df_modelagem.merge(df_scores, on="subject_id", how="left")

# 📥 Número de admissões na ICU e tempo de permanência na ICU
query_icu_stats = """
WITH icu_stats AS (
    SELECT 
        subject_id,
        COUNT(*) AS num_admissoes_icu,
        SUM(EXTRACT(EPOCH FROM (outtime - intime)) / 3600) AS tempo_total_icu_horas
    FROM mimiciv_icu.icustays
    GROUP BY subject_id
)
SELECT 
    pm.subject_id,
    COALESCE(icu.num_admissoes_icu, 0) AS num_admissoes_icu,
    COALESCE(icu.tempo_total_icu_horas, 0) AS tempo_total_icu_horas
FROM pacientes_mortos pm
LEFT JOIN icu_stats icu ON pm.subject_id = icu.subject_id;
"""

# Carregar os resultados da consulta SQL
df_icu_stats = pd.read_sql(query_icu_stats, conn)

# 🔗 Merge com o DataFrame principal
df_modelagem = df_modelagem.merge(df_icu_stats, on="subject_id", how="left")

# Preencher NaNs com 0 (caso algum paciente não tenha registros na ICU)
df_modelagem["num_admissoes_icu"] = df_modelagem["num_admissoes_icu"].fillna(0).astype(int)
df_modelagem["tempo_total_icu_horas"] = df_modelagem["tempo_total_icu_horas"].fillna(0)

# 🔧 Final: preencher NaNs
df_modelagem.fillna(0, inplace=True)

# Remover duplicatas no DataFrame com base no 'subject_id'
df_modelagem = df_modelagem.drop_duplicates(subset=['subject_id'])

# 💾 Exportar CSV
df_modelagem.to_csv("dataset_modelagem_completo.csv", index=False)
print("📁 Dataset final salvo como: dataset_modelagem_completo.csv")

/tmp/ipykernel_167189/2908548875.py:89: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_exames = pd.read_sql(query_exames, conn)
/tmp/ipykernel_167189/2908548875.py:97: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_modelagem = pd.read_sql(query_target, conn)
/tmp/ipykernel_167189/2908548875.py:105: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_demo = pd.read_sql(query_demo, conn)
/tmp/ipykernel_167189/2908548875.py:113: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string UR

📁 Dataset final salvo como: dataset_modelagem_completo.csv


In [2]:
import psycopg2
import pandas as pd

# 🔌 Conectar ao banco de dados
conn = psycopg2.connect(
    dbname="mimiciv",
    user="desafio_mimic",
    password="desafio_mimic",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

# ⚙️ Tabelas temporárias: internação e mortalidade
sql_script = """
CREATE TEMP TABLE ultima_uti AS
SELECT *
FROM (
    SELECT 
        i.subject_id,
        i.hadm_id,
        i.stay_id,
        i.intime,
        i.outtime,
        p.dod,
        a.admittime,
        a.dischtime,
        a.hospital_expire_flag,
        ROW_NUMBER() OVER (PARTITION BY i.subject_id ORDER BY i.outtime DESC) AS rn
    FROM mimiciv_icu.icustays i
    JOIN mimiciv_hosp.admissions a ON i.hadm_id = a.hadm_id
    JOIN mimiciv_hosp.patients p ON i.subject_id = p.subject_id
) t
WHERE rn = 1;

CREATE TEMP TABLE pacientes_mortos AS
SELECT
  subject_id,
  stay_id,
  CASE
    WHEN dod IS NOT NULL AND dod <= outtime THEN 'Óbito intra-UTI'
    WHEN dod IS NOT NULL AND dod > outtime AND dod <= dischtime THEN 'Óbito hospitalar pós-UTI'
    WHEN dod IS NOT NULL AND dod > dischtime AND dod <= dischtime + INTERVAL '1 year' THEN 'Óbito em até 1 ano após alta hospitalar'
    ELSE 'Sobrevivente (≥ 1 ano pós-alta)'
  END AS categoria
FROM ultima_uti;
"""
cursor.execute("ROLLBACK")
cursor.execute(sql_script)
conn.commit()

# 📥 Exames: Obter a média, variância e o último exame
query_exames = """
WITH exames_filtrados AS (
    SELECT
        e.subject_id,
        e.valuenum,
        e.charttime,
        ROW_NUMBER() OVER (PARTITION BY e.subject_id ORDER BY e.charttime DESC) AS rn
    FROM mimiciv_icu.chartevents e
    JOIN pacientes_mortos pm ON e.stay_id = pm.stay_id
    WHERE e.valuenum IS NOT NULL
),
estatisticas_exames AS (
    SELECT 
        subject_id,
        AVG(valuenum) AS media,
        VARIANCE(valuenum) AS variancia
    FROM exames_filtrados
    GROUP BY subject_id
),
ultimo_exame AS (
    SELECT 
        subject_id,
        valuenum AS ultimo_valor
    FROM exames_filtrados
    WHERE rn = 1
)
SELECT 
    e.subject_id,
    e.media,
    e.variancia,
    u.ultimo_valor
FROM estatisticas_exames e
JOIN ultimo_exame u ON e.subject_id = u.subject_id;
"""

# Carregar os resultados da consulta SQL
df_exames = pd.read_sql(query_exames, conn)

# 📥 Target
query_target = """
SELECT subject_id, stay_id,
  CASE WHEN categoria = 'Sobrevivente (≥ 1 ano pós-alta)' THEN 0 ELSE 1 END AS target
FROM pacientes_mortos;
"""
df_modelagem = pd.read_sql(query_target, conn)

# 📥 Demográficos
query_demo = """
SELECT DISTINCT p.subject_id, p.gender, p.anchor_age AS idade
FROM mimiciv_hosp.patients p
JOIN pacientes_mortos pm ON p.subject_id = pm.subject_id;
"""
df_demo = pd.read_sql(query_demo, conn)

# 📥 Charlson
query_charlson = """
SELECT subject_id, charlson_comorbidity_index AS charlson
FROM mimiciv_derived.charlson
WHERE subject_id IN (SELECT subject_id FROM pacientes_mortos);
"""
df_charlson = pd.read_sql(query_charlson, conn).drop_duplicates('subject_id')

# 🔗 Merge demográficos + Charlson
df_modelagem = df_modelagem.merge(df_demo, on="subject_id", how="left")
df_modelagem = df_modelagem.merge(df_charlson, on="subject_id", how="left")
df_modelagem["charlson"] = df_modelagem["charlson"].fillna(0)

# 📥 Diagnósticos mais frequentes
query_top_diag = """
SELECT d.icd_code, COUNT(*) AS freq
FROM mimiciv_hosp.diagnoses_icd d
JOIN (SELECT DISTINCT subject_id, hadm_id FROM mimiciv_icu.icustays) icu
  ON d.subject_id = icu.subject_id AND d.hadm_id = icu.hadm_id
JOIN pacientes_mortos pm ON d.subject_id = pm.subject_id
WHERE pm.categoria != 'Sobrevivente (≥ 1 ano pós-alta)'
GROUP BY d.icd_code
ORDER BY freq DESC
LIMIT 20;
"""
df_top_diag = pd.read_sql(query_top_diag, conn)
top_icds = df_top_diag["icd_code"].tolist()

# 📥 Diagnósticos por paciente
query_dx = """
SELECT DISTINCT subject_id, icd_code
FROM mimiciv_hosp.diagnoses_icd
WHERE subject_id IN (SELECT subject_id FROM pacientes_mortos);
"""
df_dx = pd.read_sql(query_dx, conn)

# 🔢 Criar flags binárias para diagnósticos
df_dx_flags = df_dx[["subject_id"]].drop_duplicates().copy()
for icd in top_icds:
    col = f"dx_{icd}"
    df_dx_flags[col] = df_dx["icd_code"].eq(icd).groupby(df_dx["subject_id"]).transform("max")
df_dx_flags = df_dx_flags.drop_duplicates("subject_id")
df_modelagem = df_modelagem.merge(df_dx_flags, on="subject_id", how="left")

# 🔧 Ajuste final de colunas dx_ (tira espaços e transforma em int)
df_modelagem.columns = df_modelagem.columns.str.strip()
dx_cols = [col for col in df_modelagem.columns if col.startswith("dx_")]
df_modelagem[dx_cols] = df_modelagem[dx_cols].fillna(0).astype(int)

# 🔧 Merge de exames com o dataset final
df_modelagem = df_modelagem.merge(df_exames, on="subject_id", how="left")

# 📥 Vasopressores
query_vaso = """
WITH vaso_flag AS (
    SELECT
        v.stay_id,
        MAX(CASE WHEN v.dopamine IS NOT NULL THEN 1 ELSE 0 END) AS vaso_dopamine,
        MAX(CASE WHEN v.epinephrine IS NOT NULL THEN 1 ELSE 0 END) AS vaso_epinephrine,
        MAX(CASE WHEN v.norepinephrine IS NOT NULL THEN 1 ELSE 0 END) AS vaso_norepi,
        MAX(CASE WHEN v.phenylephrine IS NOT NULL THEN 1 ELSE 0 END) AS vaso_pheny,
        MAX(CASE WHEN v.dobutamine IS NOT NULL THEN 1 ELSE 0 END) AS vaso_dobutamine,
        MAX(CASE WHEN v.milrinone IS NOT NULL THEN 1 ELSE 0 END) AS vaso_milrinone
    FROM mimiciv_derived.vasoactive_agent v
    GROUP BY v.stay_id
),
vasopressin_flag AS (
    SELECT DISTINCT stay_id, 1 AS vaso_vasopressin
    FROM mimiciv_derived.vasopressin
)
SELECT 
    COALESCE(vf.stay_id, vp.stay_id) AS stay_id,
    COALESCE(vf.vaso_dopamine, 0) AS vaso_dopamine,
    COALESCE(vf.vaso_epinephrine, 0) AS vaso_epinephrine,
    COALESCE(vf.vaso_norepi, 0) AS vaso_norepi,
    COALESCE(vf.vaso_pheny, 0) AS vaso_pheny,
    COALESCE(vf.vaso_dobutamine, 0) AS vaso_dobutamine,
    COALESCE(vf.vaso_milrinone, 0) AS vaso_milrinone,
    COALESCE(vp.vaso_vasopressin, 0) AS vaso_vasopressin
FROM vaso_flag vf
FULL OUTER JOIN vasopressin_flag vp ON vf.stay_id = vp.stay_id;
"""
df_vaso = pd.read_sql(query_vaso, conn)
df_modelagem = df_modelagem.merge(df_vaso, on="stay_id", how="left")

# 📥 Ventilação Mecânica
query_vm = """
WITH ventilacao_mec AS (
    SELECT DISTINCT stay_id, 1 AS usou_vm
    FROM mimiciv_derived.ventilation
    WHERE ventilation_status IN ('InvasiveVent', 'NonInvasiveVent', 'SupplementalOxygen', 'HFNC', 'Tracheostomy')
)
SELECT 
    u.stay_id,
    COALESCE(v.usou_vm, 0) AS usou_vm
FROM ultima_uti u
LEFT JOIN ventilacao_mec v ON u.stay_id = v.stay_id;
"""
df_vm = pd.read_sql(query_vm, conn)
df_modelagem = df_modelagem.merge(df_vm, on="stay_id", how="left")

# 📥 TSR
query_tsr = """
SELECT DISTINCT stay_id, 1 AS usou_tsr
FROM mimiciv_derived.first_day_rrt
WHERE dialysis_present = 1;
"""
df_tsr = pd.read_sql(query_tsr, conn)
df_modelagem = df_modelagem.merge(df_tsr, on="stay_id", how="left")

# 📥 SAPSII, OASIS, SOFA
query_scores = """
SELECT 
    p.subject_id,
    MAX(s.sapsii) AS sapsii,
    MAX(s.sapsii_prob) AS sapsii_prob,
    MAX(o.oasis) AS oasis,
    MAX(o.oasis_prob) AS oasis_prob,
    MAX(f.sofa_24hours) AS sofa_24hours
FROM mimiciv_icu.icustays p
JOIN mimiciv_derived.sapsii s ON p.stay_id = s.stay_id
JOIN mimiciv_derived.oasis o ON p.stay_id = o.stay_id
JOIN mimiciv_derived.sofa f ON p.stay_id = f.stay_id
JOIN pacientes_mortos pm ON p.stay_id = pm.stay_id
GROUP BY p.subject_id;
"""
df_scores = pd.read_sql(query_scores, conn)
df_modelagem = df_modelagem.merge(df_scores, on="subject_id", how="left")

# 📥 Número de admissões na ICU, tempo de permanência na ICU, número de diagnósticos e número de procedimentos
# 📥 Número de diagnósticos e procedimentos
query_icu_stats = """
WITH icu_stats AS (
    SELECT 
        subject_id,
        COUNT(*) AS num_admissoes_icu,
        SUM(EXTRACT(EPOCH FROM (outtime - intime)) / 3600) AS tempo_total_icu_horas
    FROM mimiciv_icu.icustays
    GROUP BY subject_id
),
num_diag AS (
    SELECT 
        subject_id, 
        COUNT(DISTINCT icd_code) AS num_diag
    FROM mimiciv_hosp.diagnoses_icd
    GROUP BY subject_id
),
num_proc AS (
    SELECT 
        subject_id, 
        COUNT(DISTINCT itemid) AS num_proc  -- Alterado para 'itemid' de 'procedureevents'
    FROM mimiciv_icu.procedureevents
    GROUP BY subject_id
)
SELECT 
    pm.subject_id,
    COALESCE(icu.num_admissoes_icu, 0) AS num_admissoes_icu,
    COALESCE(icu.tempo_total_icu_horas, 0) AS tempo_total_icu_horas,
    COALESCE(nd.num_diag, 0) AS num_diag,
    COALESCE(np.num_proc, 0) AS num_proc
FROM pacientes_mortos pm
LEFT JOIN icu_stats icu ON pm.subject_id = icu.subject_id
LEFT JOIN num_diag nd ON pm.subject_id = nd.subject_id
LEFT JOIN num_proc np ON pm.subject_id = np.subject_id;
"""

# Carregar os resultados da consulta SQL
df_icu_stats = pd.read_sql(query_icu_stats, conn)

# 🔗 Merge com o DataFrame principal
df_modelagem = df_modelagem.merge(df_icu_stats, on="subject_id", how="left")

# Preencher NaNs com 0 (caso algum paciente não tenha registros na ICU)
df_modelagem["num_admissoes_icu"] = df_modelagem["num_admissoes_icu"].fillna(0).astype(int)
df_modelagem["tempo_total_icu_horas"] = df_modelagem["tempo_total_icu_horas"].fillna(0)
df_modelagem["num_diag"] = df_modelagem["num_diag"].fillna(0).astype(int)
df_modelagem["num_proc"] = df_modelagem["num_proc"].fillna(0).astype(int)

# 🔧 Final: preencher NaNs
df_modelagem.fillna(0, inplace=True)

# Remover duplicatas no DataFrame com base no 'subject_id'
df_modelagem = df_modelagem.drop_duplicates(subset=['subject_id'])

# 💾 Exportar CSV
df_modelagem.to_csv("dataset_modelagem_completo.csv", index=False)
print("📁 Dataset final salvo como: dataset_modelagem_completo.csv")


/tmp/ipykernel_168341/75131249.py:89: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_exames = pd.read_sql(query_exames, conn)
/tmp/ipykernel_168341/75131249.py:97: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_modelagem = pd.read_sql(query_target, conn)
/tmp/ipykernel_168341/75131249.py:105: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_demo = pd.read_sql(query_demo, conn)
/tmp/ipykernel_168341/75131249.py:113: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sql

📁 Dataset final salvo como: dataset_modelagem_completo.csv
